# Task 3: Forecasting Future Market Trends
### This notebook will:

1. Load the trained LSTM models.
2. Generate 6–12 months of future stock price forecasts for each ticker.
3. Visualize and analyze the forecasts.

### Import necessary libraries

In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import datetime as dt

### Define paths to Models and Data folders

In [2]:
os.chdir(r'c:\users\ermias.tadesse\10x\Predictive-Analysis-And-Modeling')

# Define paths to Models and Data folders
data_folder_path = 'Data/'
models_folder_path = 'Models/'

### Load Processed Data and Models
This step loads the processed data (for comparison with forecasts) and the trained models saved in Task 2.

In [3]:
# Define tickers and load models
tickers = ['TSLA', 'BND', 'SPY']
models = {}
scalers = {}

# Load models and scalers for each ticker
for ticker in tickers:
    # Load model
    model_path = os.path.join(models_folder_path, f"{ticker}_lstm_model.h5")
    models[ticker] = load_model(model_path)
    print(f"Loaded model for {ticker}.")

    # Load processed data (for scaler and historical comparison)
    processed_data_path = os.path.join(data_folder_path, 'processed', f'{ticker}_train.csv')
    data = pd.read_csv(processed_data_path, parse_dates=['Date'], index_col='Date')
    
    # Initialize scaler and fit on full historical data
    scaler = MinMaxScaler(feature_range=(0, 1))
    close_prices = data['Close'].values.reshape(-1, 1)
    scaler.fit(close_prices)
    scalers[ticker] = scaler


Loaded model for TSLA.
Loaded model for BND.
Loaded model for SPY.


### Step 3: Generate Future Forecasts
For each ticker, we generate a 6–12 month (e.g., 180–252 trading days) forecast by feeding the LSTM model a sequence of recent data.

In [4]:
# Define forecast length (number of days)
forecast_days = 252  # Approximately one year of trading days

# Dictionary to store forecasts
forecast_results = {}

for ticker in tickers:
    print(f"\nGenerating forecast for {ticker}...")
    
    # Load latest data for forecasting
    processed_data_path = os.path.join(data_folder_path, 'processed', f'{ticker}_train.csv')
    data = pd.read_csv(processed_data_path, parse_dates=['Date'], index_col='Date')
    
    # Use the last sequence of data points for forecasting
    last_sequence = data['Close'].values[-60:].reshape(-1, 1)
    last_sequence_scaled = scalers[ticker].transform(last_sequence)
    forecast = []
    
    # Generate forecast for each day
    for _ in range(forecast_days):
        # Prepare input data
        X_input = last_sequence_scaled.reshape(1, last_sequence_scaled.shape[0], 1)
        
        # Predict and add forecasted value
        predicted_price_scaled = models[ticker].predict(X_input)
        forecast.append(predicted_price_scaled[0, 0])
        
        # Update last sequence with predicted value
        last_sequence_scaled = np.append(last_sequence_scaled[1:], predicted_price_scaled, axis=0)

    # Inverse transform the forecast
    forecast = scalers[ticker].inverse_transform(np.array(forecast).reshape(-1, 1))
    forecast_results[ticker] = forecast



Generating forecast for TSLA...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━